In [14]:
import gensim.models
import gensim.corpora
import gensim as gs
import pandas as pd
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser
from joblib import dump, load
import numpy as np
import nltk
from nltk.stem.porter import *
import numpy as np
import spacy
nlp = spacy.load("en_core_web_lg")
stemmer = nltk.stem.SnowballStemmer('english')

In [46]:
def split_punct(text):
    replacement = [(".", " . "), (",", " , "), ("!", " ! "), ("?", " ? ")]
    for k, v in replacement: 
        text = text.replace(k, v)
    return text
    
def get_emotion_words(stems, list_of_lexica):
    emotion_words = np.zeros(4)
    for index, lexicon in enumerate(list_of_lexica): 
        for stem in stems:
            if stem in lexicon:
                emotion_words[index] = emotion_words[index] + 1
    return emotion_words

def get_cons_punct_count(pos):
    cons_punct_count = 0
    for index, item in enumerate(pos[:-1]):
        if item == 97 and item == pos[index+1]:
            cons_punct_count += 1
    return cons_punct_count

def extract_features(list_of_lexica, input_message, seq_len): 
    doc = nlp(split_punct(input_message))
    doc = nlp(" ".join([token.text for token in doc if not token.is_stop and token.pos != 103]))
    if len(doc) != 0:
        pos = [token.pos for token in doc]
        stems = [stemmer.stem(token.text) for token in doc if token.pos != 97]
        emotion_words = get_emotion_words(stems, list_of_lexica)
        feature_vec = [
            (len(doc)/seq_len), (sum([token.text.isupper() for token in doc])/len(doc)), 
            (len(doc.ents)/len(doc)),get_cons_punct_count(pos), 
            emotion_words[0]/len(doc), emotion_words[1]/len(doc), emotion_words[2]/len(doc), emotion_words[3]/len(doc)]
        return feature_vec, pos, stems
    return [], [], []

In [51]:
lexica_names = ["clean_happiness", "clean_sadness", "clean_anger", "clean_fear"]
list_of_lexica = [pd.read_csv("../../lexica/" + dataset_name + ".csv") for dataset_name in lexica_names]
seq_len = {"norm_tweet": 81, "norm_emotion": 32}
dataset_name = "norm_tweet"
feature_set_name = "full"

sent = "ok , son bath , gon na find quick quotes . Gon na miss ya'll weekend . gettin sad"
fvec = extract_features(list_of_lexica, sent, seq_len[dataset_name])
for a in fvec: print(a)


rf = clf = load("../models/random_forests/" + dataset_name + "_" + feature_set_name + "_random_forests.joblib") 
print("\n",rf.predict([fvec[0]]))

[0.2345679012345679, 0.0, 0.05263157894736842, 0, 0.0, 0.0, 0.0, 0.0]
[91, 97, 92, 92, 97, 100, 94, 100, 84, 92, 97, 100, 94, 100, 96, 92, 97, 91, 84]
['ok', 'son', 'bath', 'gon', 'na', 'find', 'quick', 'quot', 'gon', 'na', 'miss', "ya'll", 'weekend', 'gettin', 'sad']

 [2]
